In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os

from graph_utils import graph_gen, find_alpha_lower_bound
from problem import Synthetic

from algorithms import IPLUX, UDC
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [ ]:
# generate problem

# network

num_node = 5
num_edge = 7

np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

# problem data

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

In [3]:
# load instance 
instance_name = 'instance1_N20E40'
instance_dir = 'instance/' + instance_name

# network
network = np.load(instance_dir + '/graph/subgraph_W.npy')
num_node = network.shape[0]
print(network.shape)

# problem data
parameters = {}
parameters['N'] = num_node
parameters['d'] = 1 # temporary, reset in load()
parameters['m'] = 1 # temporary
parameters['p'] = 1 # temporary

prob = Synthetic(parameters)
prob.save_dir = instance_dir + '/problem'
prob.load()

alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(20, 20)
loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -1.7937811849951981
x* [[ 1.69192667e-08 -4.54451916e-02 -1.25326407e-02]
 [-1.97438972e-11  2.94708451e-10  1.96457124e-01]
 [-1.55498077e-01 -1.81713964e-10 -4.30739243e-02]
 [ 2.21417334e-10  6.41377197e-11 -2.57323583e-01]
 [ 5.46491556e-01 -3.09078549e-01 -2.25841238e-10]
 [-7.95790832e-10  4.77051274e-01 -9.77296145e-02]
 [ 2.30311170e-01  9.67073127e-10 -4.08278630e-02]
 [ 4.41710887e-10  5.03547130e-01  2.55567666e-02]
 [-4.78912681e-01 -1.55547314e-09  3.47456666e-10]
 [ 1.16595532e-10 -9.51339774e-11 -4.60428545e-01]
 [ 4.42020248e-10 -4.19241406e-01  1.76861413e-01]
 [-1.46017592e-09  6.03865377e-10 -1.04286398e-09]
 [-3.70053317e-01  6.45150319e-01  3.92380391e-10]
 [ 1.95357925e-10 -2.86013131e-10 -4.44910108e-10]
 [-1.46178792e-08 -3.47711897e-01  8.41473423e-10]
 [-8.31629811e-10 -1.66205909e-01  1.02254197e

In [3]:
MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
rho_choice = [0.1, 0.5, 2, 5]
alpha_choice = [alpha_lower+6]
rho_choice = [2]


np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
        for i in range(MAX_ITER):
            alg.step()

self.prob minimize param837 @ var836 + norm1(var836) + 4.5 @ QuadForm(var836 + -param838, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) + 0.25 @ quad_over_lin(param839 @ var836, 1.0) + param840 @ var836 + param841[0] @ QuadForm(var836, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) + Promote(-2.0, (3,)) @ param842 @ var836
subject to QuadForm(var836 + -param844, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) <= param845[0]
reset


ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (5,)

In [4]:
MAX_ITER = 100

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alpha_choice = [0, 2, 4, 8, 10]
rho_choice = [1, 0.5, 2, 4, 8]
# alpha_choice = [0]
# rho_choice = [2]
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        # alg = UDC(prob, network, rho=rho, alpha=alpha, param_setting='proximal_tracking', verbose=False)
        # for i in range(MAX_ITER):
        #     alg.step()
            
        alg = UDC(prob, network, rho=rho, alpha=alpha, param_setting='PEXTRA', verbose=False)
        for i in range(MAX_ITER):
            alg.step()


UDC setting: proximal_tracking
self.prob minimize quad_over_lin(param1042 @ var1041, 1.0) + param1043 @ var1041 + param1044[0] @ norm1(var1041) + 0.0 @ quad_over_lin(Promote(param1045[0], (3,)) @ var1041 + -param1046, 1.0) + 0.5 @ quad_over_lin(maximum(param1047 + quad_over_lin(var1041 + -param1049, 1.0) + -param1050, 0.0), 1.0) + 0.5 @ quad_over_lin(param1048 + param1053 @ var1041, 1.0)
subject to QuadForm(var1041 + -param1051, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) <= param1052[0]
reset
UDC_pt alpha 0 rho 2, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

UDC_pt alpha 0 rho 2, iter 100, obj err: 2.51e-02, cons vio: 6.61e-02
time 43.68, saved

UDC setting: PEXTRA
self.prob minimize quad_over_lin(param1230 @ var1229, 1.0) + param1231 @ var1229 + param1232[0] @ norm1(var1229) + 0.0 @ quad_over_lin(Promote(param1233[0], (3,)) @ var1229 + -param1234, 1.0) + 0.5 @ quad_over_lin(maximum(param1235 + quad_over_lin(var1229

In [26]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |

from plot_utils import MyFigure

# alg_name = 'IPLUX'
alg_name = 'UDC_pt'

# alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
# rho_choice = [0.1, 0.5, 2, 5, 10]
# rho_choice = [0.05, 0.1, 0.5, 2, 5]
# MAX_ITER = 2000
alpha_choice = [0, 2, 4, 8, 10]
# rho_choice = [0.5, 1, 2, 3]
rho_choice = [2]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
# marker = [".", "o", "^", "s", "p", "P", "*"]
linestyle = ['-', '--', '-.', ':', '']



# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$|\sum_{i=1}^N f_i(x_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)|$',
                            # xlabel = 'iteration'
                            ylabel='objective error',
                            yscale='log')

cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')

x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='x distance',
                            yscale='log')

# obj_err_figure.add_line("IPLUX", alg.obj_err_log)
for rho in rho_choice:
    for alpha in alpha_choice:
        ci = alpha_choice.index(alpha)
        lsi = rho_choice.index(rho)
        # ci = rho_choice.index(rho)
        # lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'{alg_name}_a{alpha}_r{rho}'
        filename = f'log/N{num_node}/{prefix}_oe.txt'
        
        obj_err_figure.add_line_file(prefix, filename, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename, style=c+ls)
        
obj_err_figure.paint(MAX_ITER=MAX_ITER)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'